In [31]:
from scipy.stats import uniform, randint
from skopt.space import Real, Categorical
from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split
from skopt import BayesSearchCV

In [32]:
iris = load_iris()
X = iris.data
y = iris.target

In [33]:
model = RandomForestClassifier()

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X, y)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]},
             verbose=2)

In [34]:
print("Наилучшие гиперпараметры:", grid_search.best_params_)
print("Лучшее значение точности:", grid_search.best_score_)

Наилучшие гиперпараметры: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}
Лучшее значение точности: 0.9666666666666668


In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

model = RandomForestClassifier()

param_space = {
    'n_estimators': randint(10, 1000),
    'max_depth': randint(1, 100),
    'min_samples_split': uniform(0.01, 0.99),
    'min_samples_leaf': uniform(0.01, 0.49)
}

random_search = RandomizedSearchCV(estimator=model, param_distributions=param_space, n_iter=100, cv=5, random_state=42,
                                   n_jobs=-1)

random_search.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001C6BD454C10>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001C6BD634F10>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001C6BD455290>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001C69AC080D0>},
                   random_state=42)

In [36]:
best_params = random_search.best_params_
print("Наилучшие гиперпараметры:", best_params)
best_score = random_search.best_score_
print("Наилучшее значение метрики:", best_score)
test_score = random_search.score(X_test, y_test)
print("Оценка модели на тестовых данных:", test_score)

Наилучшие гиперпараметры: {'max_depth': 30, 'min_samples_leaf': 0.11404616423235531, 'min_samples_split': 0.19000671753502962, 'n_estimators': 286}
Наилучшее значение метрики: 0.9400000000000001
Оценка модели на тестовых данных: 0.98


In [37]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=42)

model = SVC()

param_space = {
    'C': Real(0.1, 10.0, 'log-uniform'),
    'gamma': Real(0.1, 5.0, 'log-uniform'),
    'kernel': Categorical(['linear', 'poly', 'rbf', 'sigmoid'])
}

opt = BayesSearchCV(model, param_space, n_iter=50, cv=3, n_jobs=-1, optimizer_kwargs={'base_estimator': 'GP'}, random_state=42)
opt.fit(X_train, y_train)

BayesSearchCV(cv=3, estimator=SVC(), n_jobs=-1,
              optimizer_kwargs={'base_estimator': 'GP'}, random_state=42,
              search_spaces={'C': Real(low=0.1, high=10.0, prior='log-uniform', transform='normalize'),
                             'gamma': Real(low=0.1, high=5.0, prior='log-uniform', transform='normalize'),
                             'kernel': Categorical(categories=('linear', 'poly', 'rbf', 'sigmoid'), prior=None)})

In [38]:
print("Best params:", opt.best_params_)
print("Best score:", opt.best_score_)

Best params: OrderedDict([('C', 0.7756486208064789), ('gamma', 3.6381617755817937), ('kernel', 'linear')])
Best score: 0.975


In [39]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=42)

model = SVC()

param_space = {
    'C': (0.1, 10.0, 'log-uniform'),
    'gamma': (0.1, 5.0, 'log-uniform'),
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
}

opt = BayesSearchCV(model, param_space, n_iter=50, cv=3, n_jobs=-1, random_state=42)
opt.fit(X_train, y_train)

BayesSearchCV(cv=3, estimator=SVC(), n_jobs=-1, random_state=42,
              search_spaces={'C': (0.1, 10.0, 'log-uniform'),
                             'gamma': (0.1, 5.0, 'log-uniform'),
                             'kernel': ['linear', 'poly', 'rbf', 'sigmoid']})

In [40]:
print("Best params:", opt.best_params_)
print("Best score:", opt.best_score_)

Best params: OrderedDict([('C', 0.7756486208064789), ('gamma', 3.6381617755817937), ('kernel', 'linear')])
Best score: 0.975
